### **Importing Libraries**

In [1]:
import torch
import math
import copy
import torch.nn as nn
import torch.nn.functional as F
import nltk
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from gensim.models import Word2Vec
import numpy as np
import torch.optim as optim
from torch.autograd import Variable
import string


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


cuda


### **Importing and Cleaning Dataset**

In [2]:
with open('./sample_data/Auguste_Maquet.txt', 'r', encoding='utf-8') as file:
    corpus = file.read()

print("Dataset Loaded")

corpus = corpus.lower()
clean_text = sent_tokenize(corpus)
translator = str.maketrans('', '', string.punctuation)
clean_text = [sentence.translate(translator) for sentence in clean_text]
# print(len(clean_text))

Dataset Loaded


### **Tokenization and Emmbedding**

In [3]:
tokenized_corpus = [word_tokenize(sentence) for sentence in clean_text]
word_to_ind = {}
longest_seq = 1
for i in range(len(tokenized_corpus)):
    token_arr = tokenized_corpus[i]
    longest_seq = max(longest_seq, len(token_arr))

    #Vocabulary
    for tokken in token_arr:
        if tokken not in word_to_ind:
            word_to_ind[tokken] = len(word_to_ind)

    token_arr = ['<sos>'] * 5 + token_arr + ['<eos>'] * 5
    tokenized_corpus[i] = token_arr

# print(tokenized_corpus[2])
word_to_ind["<sos>"] = len(word_to_ind)
word_to_ind["<eos>"] = len(word_to_ind)
print(len(word_to_ind))


word2vec_model = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4)

26161


### **Test-Train Split**

In [4]:
train_val_data, test_data = train_test_split(tokenized_corpus, test_size=0.2)

train_data, validation_data = train_test_split(train_val_data, test_size=0.125)

# Print the sizes of each set
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(validation_data)}")
print(f"Test data size: {len(test_data)}")


Training data size: 24571
Validation data size: 3511
Test data size: 7021


## **Encoder**

### **Positional Encoding**

In [5]:
class PosEncoding(nn.Module):
    def __init__(self, model_dim, max_len):
        super(PosEncoding, self).__init__()
        pos_code = torch.zeros(max_len, model_dim).to(device)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1).to(device)
        scale = torch.exp(torch.arange(0, model_dim, 2, dtype=torch.float) *
                          -(math.log(10000.0) / model_dim)).to(device)

        pos_code[:, 0::2] = torch.sin(pos * scale)
        pos_code[:, 1::2] = torch.cos(pos * scale)

        # Registering the positional encoding matrix as a buffer to avoid updating during training
        self.register_buffer('pos_code', pos_code.unsqueeze(0))

    def forward(self, inp):
        seq_len = inp.size(1)
        inp = inp.to(device) + self.pos_code[:, :seq_len]

        return inp

### **Multi Head Attention**

In [6]:
# class MultiHeadAttention(nn.Module):
#     def __init__(self, model_dim, num_layers):
#         super(MultiHeadAttention, self).__init__()
#         self.model_dim = model_dim
#         self.num_layers = num_layers
#         self.dim_key = self.model_dim // self.num_layers

#         # Linear layers for query, key, and value
#         self.query = nn.Linear(model_dim, model_dim).to(device)
#         self.key = nn.Linear(model_dim, model_dim).to(device)
#         self.value = nn.Linear(model_dim, model_dim).to(device)
#         self.out = nn.Linear(model_dim, model_dim).to(device)

#     def attention_val(self, Q, K, V, mask=None):
#         score = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.dim_key)

#         if mask is not None:
#             score = score.masked_fill(mask == 0, -1e9)

#         attn_weight = torch.softmax(score, dim=-1)

#         new_val = torch.matmul(attn_weight, V)
#         return new_val

#     def split_layers(self, x):
#         batch_size, seq_len, model_dim = x.size()
#         return x.view(batch_size, seq_len, self.num_layers, self.dim_key).transpose(1, 2)

#     def combine_layers(self, x):
#         batch_size, _, seq_len, dim_key = x.size()
#         return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.model_dim)

#     def forward(self, Q, K, V, mask=None):
#         # Split into multiple heads
#         Q = self.split_layers(self.query(Q).to(device))
#         K = self.split_layers(self.key(K).to(device))
#         V = self.split_layers(self.value(V).to(device))

#         layer_out = self.attention_val(Q, K, V, mask)
#         final_output = self.out(self.combine_layers(layer_out)).to(device)

#         return final_output


### **FeedForward**

In [7]:
# class FeedForward(nn.Module):
#     def __init__(self, model_dim, hid_dim):
#         super(FeedForward, self).__init__()
#         self.l1 = nn.Linear(model_dim, hid_dim).to(device)
#         self.ac1 = nn.ReLU().to(device)
#         self.l2 = nn.Linear(hid_dim, model_dim).to(device)

#     def forward(self, inp):
#         inp = self.l1(inp).to(device)
#         inp = self.ac1(inp).to(device)
#         inp = self.l2(inp).to(device)
#         return inp

### **Combining Encoder**

In [8]:
# class Encoder(nn.Module):
#     def __init__(self, model_dim, num_layers, hid_dim, dropout):
#         super(Encoder, self).__init__()
#         self.self_attn = MultiHeadAttention(model_dim, num_layers).to(device)
#         self.norm1 = nn.LayerNorm(model_dim).to(device)
#         self.ffn = FeedForward(model_dim, hid_dim).to(device)
#         self.norm2 = nn.LayerNorm(model_dim).to(device)
#         self.dropout = nn.Dropout(dropout).to(device)

#     def forward(self, inp, mask):

#         att_score = self.self_attn(inp, inp, inp, mask).to(device)
#         inp = self.norm1(inp + self.dropout(att_score).to(device)).to(device)
#         ffn_out = self.ffn(inp).to(device)
#         inp = self.norm2(inp + self.dropout(ffn_out).to(device)).to(device)

#         return inp

## **Decoder**

### **Combining Decoder**

In [9]:
# class Decoder(nn.Module):
#     def __init__(self, model_dim, num_layers, hid_dim, dropout):
#         super(Decoder, self).__init__()
#         self.self_attn = MultiHeadAttention(model_dim, num_layers).to(device)
#         self.norm1 = nn.LayerNorm(model_dim).to(device)
#         self.cross_attn = MultiHeadAttention(model_dim, num_layers).to(device)
#         self.norm2 = nn.LayerNorm(model_dim).to(device)
#         self.ffn = FeedForward(model_dim, hid_dim).to(device)
#         self.norm3 = nn.LayerNorm(model_dim).to(device)
#         self.dropout = nn.Dropout(dropout).to(device)

#     def forward(self, inp, encoder_out, source_mask, target_mask):

#         att_score = self.self_attn(inp, inp, inp, target_mask).to(device)
#         inp = self.norm1(inp + self.dropout(att_score).to(device)).to(device)
#         att_score_cross = self.cross_attn(inp, encoder_out, encoder_out, source_mask).to(device)
#         inp = self.norm2(inp + self.dropout(att_score_cross).to(device)).to(device)
#         ffn_out = self.ffn(inp).to(device)
#         inp = self.norm3(inp + self.dropout(ffn_out).to(device)).to(device)

#         return inp

## **Transformer**

In [10]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, model_dim, num_layer, num_heads, hid_dim, max_len, dropout=0.1, pretrained_embeddings=None):
        super(Transformer, self).__init__()
        self.model_dim = model_dim
        if pretrained_embeddings is not None:
            self.embed = nn.Embedding.from_pretrained(torch.tensor(pretrained_embeddings, dtype=torch.float), freeze=True)
        else:
            self.embed = nn.Embedding(vocab_size, model_dim)

        self.pos_enc = PosEncoding(model_dim, max_len)

        decoder_layer = nn.TransformerDecoderLayer(d_model=model_dim, nhead=num_heads, dim_feedforward=hid_dim, dropout=dropout, batch_first=True)
        self.declayer = nn.TransformerDecoder(decoder_layer, num_layers=num_layer)

        self.final_layer = nn.Linear(model_dim, vocab_size)
        self.max_len = max_len
        self.to(device)

    def forward(self, src, target):
        target = target.to(device)

        target_mask = self.generate_square_subsequent_mask(target.size(1)).to(device)

        # target_emb = self.pos_enc(self.encoderEmb(target))
        # padding_mask = (target == 0).to(device)
        target_emb = self.pos_enc(self.embed(target) / math.sqrt(self.model_dim)).to(device)

        dec_out = self.declayer(target_emb, src, tgt_mask=target_mask)

        final_out = self.final_layer(dec_out)
        return final_out

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

### **Creating Datasets**

In [11]:
class LM_Dataset(torch.utils.data.Dataset):
    def __init__(self, sentences, targets):
        self.sentences = sentences
        self.targets = targets

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        target = self.targets[idx]

        sentence_tensor = torch.tensor(sentence, dtype=torch.long)
        target_tensor = torch.tensor(target, dtype=torch.long)

        return sentence_tensor, target_tensor

### **Creating Input**

In [12]:
# def process_sentences(sentences, word_to_index, max_len=None):
#     def words_to_indices(words, word_to_index):
#         return [word_to_index.get(word, 0) for word in words]

#     context_indices = []
#     central_word_indices = []

#     for sentence in sentences:
#         word_indices = words_to_indices(sentence, word_to_index)

#         if max_len is not None:
#             word_indices = word_indices[:max_len] + [0] * (max_len - len(word_indices))

#         context_indices.append(word_indices[:-1])

#         central_word_indices.append(word_indices[1:])

#     return context_indices, central_word_indices

# train_gram_inp, train_cen_inp = process_sentences(train_data, word_to_ind, max_len=20)
# val_gram_inp, val_cen_inp = process_sentences(validation_data, word_to_ind, max_len=20)
# test_gram_inp, test_cen_inp = process_sentences(test_data, word_to_ind, max_len=20)
# print(train_cen_inp[0])

def process_sentences(sentences, word_to_index, max_len=None):
    def words_to_indices(words, word_to_index):
        return [word_to_index.get(word, 0) for word in words]

    context_indices = []
    central_word_indices = []

    for sentence in sentences:
        word_indices = words_to_indices(sentence, word_to_index)

        if max_len is not None:
            if len(word_indices) < max_len:
                word_indices = word_indices + [0] * (max_len - len(word_indices))
            else:
                word_indices = word_indices[:max_len]

        context_indices.append(word_indices[:-1])
        central_word_indices.append(word_indices[1:])

    return context_indices, central_word_indices

train_gram_inp, train_cen_inp = process_sentences(train_data, word_to_ind, max_len=20)
val_gram_inp, val_cen_inp = process_sentences(validation_data, word_to_ind, max_len=20)
test_gram_inp, test_cen_inp = process_sentences(test_data, word_to_ind, max_len=20)

# Example output
# print(train_cen_inp[0])




### **Train Model**

In [17]:
dataset_train = LM_Dataset(train_gram_inp, train_cen_inp)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=16, shuffle=True)

dataset_val = LM_Dataset(val_gram_inp, val_cen_inp)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=16)

pretrained_embeddings = word2vec_model.wv.vectors

model = Transformer(vocab_size=len(word_to_ind), model_dim=100, num_heads=4, num_layer=6, hid_dim=300, max_len=40, dropout=0.1, pretrained_embeddings=pretrained_embeddings)
model.to(device)

num_epochs = 10
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in dataloader_train:
        context_words, target_words = batch
        context_words = context_words.to(device)
        target_words = target_words.to(device)
        src = torch.zeros(context_words.size(0), 1, model.model_dim).to(context_words.device)


        optimizer.zero_grad()

        outputs = model(src, context_words)
        # Check the shapes and a few values of the outputs
        # print(f"Outputs shape: {outputs.shape}")
        # print(f"Outputs sample: {outputs[0, :5]}")  # Print first few values of the first batch

        outputs = outputs.view(-1, outputs.size(-1))
        target_words = target_words.view(-1)

        loss = criterion(outputs, target_words)
        # print(f"Loss: {loss.item()}")

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(dataloader_train)

    # Validation loop
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in dataloader_val:
            context_words, target_words = batch
            context_words = context_words.to(device)
            target_words = target_words.to(device)
            src = torch.zeros(context_words.size(0), 1, model.model_dim).to(context_words.device)



            outputs = model(src, context_words)
            outputs = outputs.view(-1, outputs.size(-1))
            target_words = target_words.view(-1)
            loss = criterion(outputs, target_words)
            total_val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += target_words.size(0)
            correct += (predicted == target_words).sum().item()


    avg_val_loss = total_val_loss / len(dataloader_val)
    accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {accuracy:.2f}%')


Epoch [1/10], Train Loss: 4.3679, Val Loss: 3.9752, Val Accuracy: 39.28%
Epoch [2/10], Train Loss: 3.8375, Val Loss: 3.8108, Val Accuracy: 40.59%
Epoch [3/10], Train Loss: 3.6723, Val Loss: 3.7316, Val Accuracy: 41.24%
Epoch [4/10], Train Loss: 3.5624, Val Loss: 3.6966, Val Accuracy: 41.75%
Epoch [5/10], Train Loss: 3.4689, Val Loss: 3.6955, Val Accuracy: 41.85%
Epoch [6/10], Train Loss: 3.4357, Val Loss: 3.8171, Val Accuracy: 40.82%
Epoch [7/10], Train Loss: 3.3803, Val Loss: 3.6750, Val Accuracy: 42.32%
Epoch [8/10], Train Loss: 3.2768, Val Loss: 3.7047, Val Accuracy: 42.39%
Epoch [9/10], Train Loss: 3.2283, Val Loss: 3.6899, Val Accuracy: 42.78%
Epoch [10/10], Train Loss: 3.1824, Val Loss: 3.7015, Val Accuracy: 43.07%


### **Evaluate Model**

In [18]:
dataset_test = LM_Dataset(test_gram_inp, test_cen_inp)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=16)

model.eval()
correct = 0
total = 0
total_loss = 0
total_tokens = 0
criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for batch in dataloader_test:
        context_words, target_words = batch
        context_words = context_words.to(device)
        target_words = target_words.to(device)
        src = torch.zeros(context_words.size(0), 1, model.model_dim).to(context_words.device)


        outputs = model(src, context_words)
        outputs = outputs.view(-1, outputs.size(-1))
        target_words = target_words.view(-1)

        loss = criterion(outputs, target_words)
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += target_words.size(0)
        correct += (predicted == target_words).sum().item()

accuracy = 100 * correct / total
average_loss = total_loss / len(dataloader_test)
perplexity = math.exp(average_loss)

print(f'Test Accuracy: {accuracy:.2f}%')
print(f'Average Test Loss: {average_loss:.4f}')
print(f'Perplexity: {perplexity:.2f}')

Test Accuracy: 43.31%
Average Test Loss: 3.6676
Perplexity: 39.16
